In [4]:
import fastf1
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
import warnings
from pathlib import Path
import seaborn as sns

warnings.filterwarnings('ignore')

fastf1.Cache.enable_cache(os.getcwd()) # replace with ur own folder path to make a cache for faster loading

# Window size (range of the sliding window used in delta angle calculation)
N = 40 # (40 + 1 + 40  = 81 data points)

# Max and Min delta angle for a data point to be classified as being part of a corner
CORNER_THRESHOLD = 0.5

In [5]:
year = 2025

gp_events = ['British Grand Prix', 'Belgian Grand Prix', 'Hungarian Grand Prix', 'Dutch Grand Prix',
              'Italian Grand Prix', 'Monaco Grand Prix']

for event in gp_events:
    session = fastf1.get_session(year, event, 'R')
    session.load()

    for driver_number in session.drivers:
        driver_data = session.get_driver(driver_number)
        driver = driver_data['Abbreviation']

        folder = f"{event}"
        dataFileName = f"{driver}.csv"
        filepath = os.path.join(folder, dataFileName)
        os.makedirs(folder, exist_ok=True)

        if os.path.exists(filepath):
            print(f"Skipping {driver} in {event}, already saved.")
            continue

        telemetry_data = session.laps.pick_driver(driver).get_telemetry()
        telemetry_data = telemetry_data[['Time', 'X', 'Y', 'Z', 'Speed']]

        laps_data = session.laps[['Time', 'LapNumber', 'LapStartTime', 'LapTime']]
        laps_data = laps_data.sort_values('LapStartTime')

        # Create an array of lap start times and lap numbers
        lap_starts = laps_data['LapStartTime'].values
        lap_ends = laps_data['Time'].values
        lap_numbers = laps_data['LapNumber'].values

        # Use searchsorted to find the lap index for each telemetry time
        idx = np.searchsorted(lap_starts, telemetry_data['Time'], side='right') - 1

        # Make sure idx is valid
        idx[idx < 0] = 0
        idx[idx >= len(lap_numbers)] = len(lap_numbers)-1

        # Assign LapNumber only if telemetry is within lap interval
        telemetry_data['LapNumber'] = np.where(
            (telemetry_data['Time'] >= lap_starts[idx]) & (telemetry_data['Time'] < lap_ends[idx]),
            lap_numbers[idx],
            np.nan)

        telemetry_data = telemetry_data.dropna(subset=['LapNumber'])
        telemetry_data['LapNumber'] = telemetry_data['LapNumber'].astype(int)

        dx1 = telemetry_data['X'] - telemetry_data['X'].shift(N)
        dy1 = telemetry_data['Y'] - telemetry_data['Y'].shift(N)
        dx2 = telemetry_data['X'].shift(-N) - telemetry_data['X']
        dy2 = telemetry_data['Y'].shift(-N) - telemetry_data['Y']

        telemetry_data['Delta_Angle'] = np.arctan2(dx1*dy2 - dy1*dx2, dx1*dx2 + dy1*dy2)
        telemetry_data['Delta_Angle'] = telemetry_data['Delta_Angle'].fillna(0)

        telemetry_data['IsCorner'] = (telemetry_data['Delta_Angle'] > CORNER_THRESHOLD) | (telemetry_data['Delta_Angle'] < -CORNER_THRESHOLD)

        telemetry_data.to_csv(filepath)

core           INFO 	Loading data for British Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '27', '44', '1', '10', '18', '23', '14', '63', '87', '55', '31', '16', '22', '12', '6', '5', '30', '43']
core           INFO 	Loading data for Belgian Grand Prix - Rac

Skipping NOR in British Grand Prix, already saved.
Skipping PIA in British Grand Prix, already saved.
Skipping HUL in British Grand Prix, already saved.
Skipping HAM in British Grand Prix, already saved.
Skipping VER in British Grand Prix, already saved.
Skipping GAS in British Grand Prix, already saved.
Skipping STR in British Grand Prix, already saved.
Skipping ALB in British Grand Prix, already saved.
Skipping ALO in British Grand Prix, already saved.
Skipping RUS in British Grand Prix, already saved.
Skipping BEA in British Grand Prix, already saved.
Skipping SAI in British Grand Prix, already saved.
Skipping OCO in British Grand Prix, already saved.
Skipping LEC in British Grand Prix, already saved.
Skipping TSU in British Grand Prix, already saved.
Skipping ANT in British Grand Prix, already saved.
Skipping HAD in British Grand Prix, already saved.
Skipping BOR in British Grand Prix, already saved.
Skipping LAW in British Grand Prix, already saved.
Skipping COL in British Grand P

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '81'
core        WARNING 	Fixed incorrect tyre stint information for driver '4'
core        WARNING 	Fixed incorrect tyre stint information for driver '16'
core        WARNING 	Fixed incorrect tyre stint information for driver '1'
core        WARNING 	Fixed incorrect tyre stint information for driver '63'
core        WARNING 	Fixed incorrect tyre stint information for driver '23'
core        WARNING 	Fixed incorrect tyre stint information for driver '44'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stin

Skipping PIA in Belgian Grand Prix, already saved.
Skipping NOR in Belgian Grand Prix, already saved.
Skipping LEC in Belgian Grand Prix, already saved.
Skipping VER in Belgian Grand Prix, already saved.
Skipping RUS in Belgian Grand Prix, already saved.
Skipping ALB in Belgian Grand Prix, already saved.
Skipping HAM in Belgian Grand Prix, already saved.
Skipping LAW in Belgian Grand Prix, already saved.
Skipping BOR in Belgian Grand Prix, already saved.
Skipping GAS in Belgian Grand Prix, already saved.
Skipping BEA in Belgian Grand Prix, already saved.
Skipping HUL in Belgian Grand Prix, already saved.
Skipping TSU in Belgian Grand Prix, already saved.
Skipping STR in Belgian Grand Prix, already saved.
Skipping OCO in Belgian Grand Prix, already saved.
Skipping ANT in Belgian Grand Prix, already saved.
Skipping ALO in Belgian Grand Prix, already saved.
Skipping SAI in Belgian Grand Prix, already saved.
Skipping COL in Belgian Grand Prix, already saved.
Skipping HAD in Belgian Grand P

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '63', '16', '14', '5', '18', '30', '1', '12', '6', '44', '27', '55', '23', '31', '22', '43', '10', '87']
core           INFO 	Loading data for Dutch Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


Skipping NOR in Hungarian Grand Prix, already saved.
Skipping PIA in Hungarian Grand Prix, already saved.
Skipping RUS in Hungarian Grand Prix, already saved.
Skipping LEC in Hungarian Grand Prix, already saved.
Skipping ALO in Hungarian Grand Prix, already saved.
Skipping BOR in Hungarian Grand Prix, already saved.
Skipping STR in Hungarian Grand Prix, already saved.
Skipping LAW in Hungarian Grand Prix, already saved.
Skipping VER in Hungarian Grand Prix, already saved.
Skipping ANT in Hungarian Grand Prix, already saved.
Skipping HAD in Hungarian Grand Prix, already saved.
Skipping HAM in Hungarian Grand Prix, already saved.
Skipping HUL in Hungarian Grand Prix, already saved.
Skipping SAI in Hungarian Grand Prix, already saved.
Skipping ALB in Hungarian Grand Prix, already saved.
Skipping OCO in Hungarian Grand Prix, already saved.
Skipping TSU in Hungarian Grand Prix, already saved.
Skipping COL in Hungarian Grand Prix, already saved.
Skipping GAS in Hungarian Grand Prix, already 

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['81', '1', '6', '63', '23', '87', '18', '14', '22', '31', '43', '30', '55', '27', '5', '12', '10', '4', '16', '44']
core           INFO 	Loading data for Italian Grand Prix - Race [v3.6.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...


Skipping PIA in Dutch Grand Prix, already saved.
Skipping VER in Dutch Grand Prix, already saved.
Skipping HAD in Dutch Grand Prix, already saved.
Skipping RUS in Dutch Grand Prix, already saved.
Skipping ALB in Dutch Grand Prix, already saved.
Skipping BEA in Dutch Grand Prix, already saved.
Skipping STR in Dutch Grand Prix, already saved.
Skipping ALO in Dutch Grand Prix, already saved.
Skipping TSU in Dutch Grand Prix, already saved.
Skipping OCO in Dutch Grand Prix, already saved.
Skipping COL in Dutch Grand Prix, already saved.
Skipping LAW in Dutch Grand Prix, already saved.
Skipping SAI in Dutch Grand Prix, already saved.
Skipping HUL in Dutch Grand Prix, already saved.
Skipping BOR in Dutch Grand Prix, already saved.
Skipping ANT in Dutch Grand Prix, already saved.
Skipping GAS in Dutch Grand Prix, already saved.
Skipping NOR in Dutch Grand Prix, already saved.
Skipping LEC in Dutch Grand Prix, already saved.
Skipping HAM in Dutch Grand Prix, already saved.


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

Skipping NOR in Monaco Grand Prix, already saved.
Skipping LEC in Monaco Grand Prix, already saved.
Skipping PIA in Monaco Grand Prix, already saved.
Skipping VER in Monaco Grand Prix, already saved.
Skipping HAM in Monaco Grand Prix, already saved.
Skipping HAD in Monaco Grand Prix, already saved.
Skipping OCO in Monaco Grand Prix, already saved.
Skipping LAW in Monaco Grand Prix, already saved.
Skipping ALB in Monaco Grand Prix, already saved.
Skipping SAI in Monaco Grand Prix, already saved.
Skipping RUS in Monaco Grand Prix, already saved.
Skipping BEA in Monaco Grand Prix, already saved.
Skipping COL in Monaco Grand Prix, already saved.
Skipping BOR in Monaco Grand Prix, already saved.
Skipping STR in Monaco Grand Prix, already saved.
Skipping HUL in Monaco Grand Prix, already saved.
Skipping TSU in Monaco Grand Prix, already saved.
Skipping ANT in Monaco Grand Prix, already saved.
Skipping ALO in Monaco Grand Prix, already saved.
Skipping GAS in Monaco Grand Prix, already saved.


In [9]:
root_directory = Path(os.getcwd())

for file in root_directory.rglob('*.csv'):
    telemetry_data = pd.read_csv(file, usecols=["X", "Y", "Delta_Angle", "IsCorner"])

    plt.figure(figsize=(6.4, 4.8))
    cmap = sns.color_palette('viridis', as_cmap = True)
    norm = plt.Normalize(vmin=telemetry_data['Delta_Angle'].min(), vmax=telemetry_data['Delta_Angle'].max())

    sc = sns.scatterplot(data=telemetry_data,
                        x='X', y='Y',
                        hue='Delta_Angle',
                        palette='viridis',
                        hue_norm=norm,
                        legend=False,
                        edgecolor=None)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax = plt.gca())

    plt.savefig(os.path.join(file.parent, f"{file.stem}_deltaAngle.png"))
    plt.close()

    plt.figure(figsize=(6.4, 4.8))
    sns.scatterplot(data=telemetry_data,
                    x='X', y='Y',
                    hue='IsCorner',
                    palette={False: "blue", True: "red"},
                    edgecolor=None)

    plt.savefig(os.path.join(file.parent, f"{file.stem}_isCorner.png"))
    plt.close()

In [ ]:
for file in root_directory.rglob('*.csv'):
    telemetry_data = pd.read_csv(file)
    lap_stats = (
        telemetry_data
        .groupby(["LapNumber", "IsCorner"])
        .agg(
            AvgSpeed=("Speed", "mean"),
            MaxSpeed=("Speed", "max"),
            MinSpeed=("Speed", "min"),
            Count=("Speed", "size")
        )
        .reset_index()
    )

    lap_stats = lap_stats.pivot(
        index="LapNumber", 
        columns="IsCorner", 
        values=["AvgSpeed", "MaxSpeed", "MinSpeed", "Count"]
    )

    lap_stats.columns = [f"{stat}_{'Corner' if corner else 'Straight'}" 
                        for stat, corner in lap_stats.columns]

    lap_stats = lap_stats.reset_index()

    lap_stats.to_csv(f"{file.parent}/{file.stem}_lapStats.csv")